# rdams_client.py Example

rdams_client.py can work as a command line script or python module to interact with python objects directly.

#### First, let's import the module and see what we can do.

In [ ]:
# Make sure this is in the same directory as this notebook or is in your PATH environment variable.
import rdams_client as rc

**We can use the query() function to make calls, or we can call the function itself.**

**Let's try to get a summary of a dataset.**

NOTE: when executing this command for the first time, you may be asked to authenticate

In [ ]:
# You might want to use query() if you are more familiar with the command line calls.
# By default, query will print the result.
# You can turn this off with the -noprint or -np option
result = rc.query(['-get_summary', 'ds633.0'])

In [ ]:
# Same thing as above, but we call the function directly
result = rc.get_summary('633.0')
print(result)

In [ ]:
# Of course, we could have always done this on the command line.
# NOTE: This cell will only work if it's the current directory
! ./rdams_client.py -get_summary '633.0'

### Example workflow

In [ ]:
dsid = 'ds083.2'
param_response = rc.query(['-get_param_summary', dsid, '-np'])

# get_param_summary() returns an RDA response object that has more info that we need, so we'll filter it out
param_data = param_response['data']['data']

# Next we will just pull out the descriptions or long name of each of the parameters.
params = list(map(lambda x: x['param_description'], param_data))
print('\n'.join(params))

**Great!**

**But, these parameters are just the long name, so let's put them in dictionary to map the long name to the actual variable name. This way, we can use the variable in a subset request.**

In [ ]:
param_map = {}
for _param in param_data:
    long_name = _param['param_description']
    short_name = _param['param']
    param_map[long_name] = short_name

for k,v in param_map.items(): print('{:7} : {}'.format(v, k))
    

### Now, let's get full metadata and filter it.

For more advanced use cases, we will need to use `get_metadata()` to get everything we need about a variable. We can manipulate the result to filter variables.

In [ ]:
# For large datasets this may take a while as you're pulling all the metadata.
metadata_response = rc.query(['-get_metadata', dsid])

**This is too much information, so let's filter it down.**

In [ ]:
# List of dicts representing a variable
_vars = metadata_response['data']['data']

# Just get temperature variables
TMP_variables = list(filter(lambda v: v['param'] == 'TMP',_vars)) 

# Let's say we're only interested in 2010
TMP_2010_variables = list(filter(
        lambda v: v['start_date'] < 201001010000 and v['end_date'] > 201101010000 ,TMP_variables
        )) 

# We only should have 1 variable
assert len(TMP_2010_variables) == 1
my_var = TMP_2010_variables[0]

# Now let's look at the levels available:
for lev in my_var['levels']:
    print('{:6} {:10} {}'.format(lev['level'], lev['level_value'],lev['level_description']))




#### But let's say I only want Isobaric surfaces between 100 and 500Hpa. 

In [ ]:
ISBL_levels = set()
ISBL_levels_printed = []
for lev in my_var['levels']:
    if lev['level_description'] == 'Isobaric surface' \
            and float(lev['level_value']) >= 100 \
            and float(lev['level_value']) <= 500:
        ISBL_levels.add(lev['level_value'])
        ISBL_levels_printed.append(lev)
for lev in ISBL_levels_printed:
    print('{:6} {:10} {}'.format(lev['level'], lev['level_value'],lev['level_description']))

### Finally, let's get make a request

In [ ]:
# Let's get a template. 
response = rc.get_control_file_template(dsid)
template = response['data']['template'] # Template string

# Parse the string
template_dict = rc.read_control_file(template)

# Insert our TMP param
template_dict['param'] = 'TMP'
template_dict['level'] = 'ISBL:' + '/'.join(ISBL_levels)
template_dict

In [ ]:
# Now We can submit a request
response = rc.submit_json(template_dict)
assert response['http_response'] == 200
print(response)

print("Success!")